# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [32]:
from xml.etree import ElementTree as ET
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [33]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [34]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [35]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [36]:
document = ET.parse( './data/mondial_database.xml' )

## 1. 10 countries with the lowest infant mortality rates:

In [37]:
# create a dataframe to hold the parsed data values
df_infant_mort = pd.DataFrame(columns=('country', 'infant_mortality'))

# parse the document
for element in document.iterfind('country'):
    try:
        # retrieve values needed 
        country = element.find('name').text
        infant_mortality = element.find('infant_mortality').text
        
        # add values to the dataframe
        row = dict(zip(['country', 'infant_mortality'], [country, infant_mortality]))
        row = pd.Series(row)
        row.name = 1
        df_infant_mort = df_infant_mort.append(row)

    except:
        pass

# change the numeric data types to float
df_infant_mort['infant_mortality'] = df_infant_mort.infant_mortality.astype(float)
df_infant_mort.head()

,country,infant_mortality
1,Albania,13.19
1,Greece,4.78
1,Macedonia,7.90
1,Serbia,6.16
1,Andorra,3.69


In [38]:
# sort by mortality rate to get the 10  lowest countries in infant mortality
df_infant_mort.sort_values(by='infant_mortality', inplace=True)

# reindex the dataframe
df_infant_mort.reset_index(drop=True, inplace=True)

# display the results
df_infant_mort.head(10)

,country,infant_mortality
0,Monaco,1.81
1,Japan,2.13
2,Bermuda,2.48
3,Norway,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


## 2. 10 cities with the largest population:

In [39]:
# create a dataframe to hold the parsed data values
df_city_pop = pd.DataFrame(columns=('country','city_id', 'city_name', 'year', 'population'))

# parse the document
for element in document.iterfind('country'):
      try:
        
        # retrieve values needed 
        country = element.find('name').text
    
        for subelement in element.getiterator('city'):
            city_name = subelement.find('name').text
            city_id = subelement.attrib['id']
            
            for pop in subelement.iterfind('population'):
                year = pop.attrib['year']
                population = pop.text
                
                # add values to the dataframe
                row = dict(zip(['country','city_id', 'city_name', 'year', 'population'], [country, city_id, city_name, year, population]))
                row = pd.Series(row)
                row.name = 1
                df_city_pop = df_city_pop.append(row)        

      except:
            pass

# change the data type of numeric columns
df_city_pop['population'] = df_city_pop.population.astype(float)
df_city_pop['year'] = df_city_pop.year.astype(int)

df_city_pop.head()

,country,city_id,city_name,year,population
1,Albania,cty-Albania-Tirane,Tirana,1987,"192,000.00"
1,Albania,cty-Albania-Tirane,Tirana,1990,"244,153.00"
1,Albania,cty-Albania-Tirane,Tirana,2011,"418,495.00"
1,Albania,stadt-Shkoder-AL-AL,Shkodër,1987,"62,000.00"
1,Albania,stadt-Shkoder-AL-AL,Shkodër,2011,"77,075.00"


In [40]:
# get the latest population for each city
# use city_id instead of city_name to avoid mixing up cities with similar names in different countries

df_last_pop = pd.DataFrame(df_city_pop.groupby('city_id')['year'].max())

df_last_pop.reset_index(level=0, inplace=True)
df_last_pop.columns = ['city_id', 'year']
df_last_pop.head()

,city_id,year
0,city-Bayamon-PR-PR,2010
1,city-Belgrade-SRB-SRB,2011
2,city-Carolina-PR-PR,2010
3,city-Dili-TL-TL,2010
4,city-EC-Cuenca,2010


In [41]:
# use the results of latest popultaion run to get the latest population of each city
df_city_pop_last = pd.merge(df_last_pop, df_city_pop, how='inner', on=['city_id', 'year'])
df_city_pop_last.sort_values('population', ascending=False, inplace=True)
df_city_pop_last.reset_index(drop=True, inplace=True)


In [42]:
# results: 10 cities with the largest population
df_city_pop_last.head(10)


,city_id,year,country,city_name,population
0,cty-China-Shanghai,2010,China,Shanghai,"22,315,474.00"
1,cty-Turkey-Istanbul,2012,Turkey,Istanbul,"13,710,512.00"
2,cty-India-2,2011,India,Mumbai,"12,442,373.00"
3,cty-Russia-Moscow,2013,Russia,Moskva,"11,979,529.00"
4,cty-China-3,2010,China,Beijing,"11,716,620.00"
5,cty-Brazil-Sao-Paulo,2010,Brazil,São Paulo,"11,152,344.00"
6,cty-China-5,2010,China,Tianjin,"11,090,314.00"
7,cty-China-8,2010,China,Guangzhou,"11,071,424.00"
8,cty-India-New-Delhi,2011,India,Delhi,"11,034,555.00"
9,cty-China-105,2010,China,Shenzhen,"10,358,381.00"


## 3. 10 ethnic groups with the largest overall populations:

In [43]:
# create a dataframe to hold the parsed data values
df_country_pop = pd.DataFrame(columns=('country', 'year', 'population'))

# parse the document
for element in document.iterfind('country'):
      try:
        
        #retrieve values needed 
        country = element.find('name').text

        for pop in element.iterfind('population'):
            year = pop.attrib['year']
            population = pop.text

            # add values to the dataframe
            row = dict(zip(['country', 'year', 'population'], [country, year, population]))
            row = pd.Series(row)
            row.name = 1
            df_country_pop = df_country_pop.append(row)        

      except:
            pass

# change the data type of numeric columns        
df_country_pop['population'] = df_country_pop.population.astype(float)
df_country_pop['year'] = df_country_pop.year.astype(int)
df_country_pop.head(20)


,country,year,population
1,Albania,1950,"1,214,489.00"
1,Albania,1960,"1,618,829.00"
1,Albania,1970,"2,138,966.00"
1,Albania,1980,"2,734,776.00"
1,Albania,1990,"3,446,882.00"
1,Albania,1997,"3,249,136.00"
1,Albania,2000,"3,304,948.00"
1,Albania,2001,"3,069,275.00"
1,Albania,2011,"2,800,138.00"
1,Greece,1861,"1,096,810.00"


In [44]:
# get the latest population for each country

df_country_last_pop = pd.DataFrame(df_country_pop.groupby('country')['year'].max())
df_country_last_pop.reset_index(level=0, inplace=True)
df_country_last_pop.columns = ['country', 'year']
df_country_last_pop.head()


,country,year
0,Afghanistan,2013
1,Albania,2011
2,Algeria,2010
3,American Samoa,2010
4,Andorra,2011


In [45]:
# use the latest population run to get the latest country populaion

df_country_population_last = pd.merge(df_country_last_pop, df_country_pop, how='inner', on=['country', 'year'])
df_country_population_last.sort_values('population', ascending=False, inplace=True)
df_country_population_last.reset_index(drop=True, inplace=True)
df_country_population_last.head(10)

,country,year,population
0,China,2013,"1,360,720,000.00"
1,India,2011,"1,210,854,977.00"
2,United States,2014,"318,857,056.00"
3,Indonesia,2014,"252,124,458.00"
4,Brazil,2014,"202,768,562.00"
5,Pakistan,2010,"173,149,306.00"
6,Nigeria,2011,"164,294,516.00"
7,Bangladesh,2011,"149,772,364.00"
8,Russia,2014,"143,666,931.00"
9,Japan,2013,"127,298,000.00"


In [46]:
# get the ethnic groups and their percentages in each country

# create a dataframe to hold the parsed data values
df_country_ethnics = pd.DataFrame(columns=('country', 'ethnic_group', 'ethnic_group_pct'))

# parse the document
for element in document.iterfind('country'):
      try:
        
        # retrieve values needed 
        country = element.find('name').text

        for eth in element.iterfind('ethnicgroup'):
            ethnic_group_pct = eth.attrib['percentage']
            ethnic_group = eth.text

            # add values to the dataframe
            row = dict(zip(['country', 'ethnic_group', 'ethnic_group_pct'], [country, ethnic_group, ethnic_group_pct]))
            row = pd.Series(row)
            row.name = 1
            df_country_ethnics = df_country_ethnics.append(row)        

      except:
            pass

# change the data type of numeric columns
df_country_ethnics['ethnic_group_pct'] = df_country_ethnics.ethnic_group_pct.astype(float)
df_country_ethnics.head(10)


,country,ethnic_group,ethnic_group_pct
1,Albania,Albanian,95.00
1,Albania,Greek,3.00
1,Greece,Greek,93.00
1,Macedonia,Macedonian,64.20
1,Macedonia,Albanian,25.20
1,Macedonia,Turkish,3.90
1,Macedonia,Gypsy,2.70
1,Macedonia,Serb,1.80
1,Serbia,Serb,82.90
1,Serbia,Montenegrin,0.90


In [47]:
# join the two data frames of ethnic group % and the latest coutry poplation
df_country_ethnics_pop = pd.merge(df_country_ethnics, df_country_population_last, how='inner', on=['country'])

# multiply the ethnic group % into the latest coutry poplation to get the ethnic group poplation
df_country_ethnics_pop['ethnic_pop'] = df_country_ethnics_pop.ethnic_group_pct * df_country_ethnics_pop.population

df_country_ethnics_pop.head(10)

,country,ethnic_group,ethnic_group_pct,year,population,ethnic_pop
0,Albania,Albanian,95.00,2011,"2,800,138.00","266,013,110.00"
1,Albania,Greek,3.00,2011,"2,800,138.00","8,400,414.00"
2,Greece,Greek,93.00,2011,"10,816,286.00","1,005,914,598.00"
3,Macedonia,Macedonian,64.20,2011,"2,059,794.00","132,238,774.80"
4,Macedonia,Albanian,25.20,2011,"2,059,794.00","51,906,808.80"
5,Macedonia,Turkish,3.90,2011,"2,059,794.00","8,033,196.60"
6,Macedonia,Gypsy,2.70,2011,"2,059,794.00","5,561,443.80"
7,Macedonia,Serb,1.80,2011,"2,059,794.00","3,707,629.20"
8,Serbia,Serb,82.90,2011,"7,120,666.00","590,303,211.40"
9,Serbia,Montenegrin,0.90,2011,"7,120,666.00","6,408,599.40"


In [48]:
# get total ethnic groups population in the whole data (regardless of the country)
df_thnics_pop = pd.DataFrame(df_country_ethnics_pop.groupby('ethnic_group')['ethnic_pop'].sum())
df_thnics_pop.reset_index(level=0, inplace=True)
df_thnics_pop.columns = ['ethnic_group', 'total_population']

# sort the dataframe by total ethnic group population  and reidex
df_thnics_pop.sort_values('total_population', ascending=False, inplace=True)
df_thnics_pop.reset_index(drop=True, inplace=True)

# remove the decimal digits
df_thnics_pop.total_population = round(df_thnics_pop.total_population)


# results: 10 ethnic groups with the largest overall populations
df_thnics_pop.head(10)


,ethnic_group,total_population
0,Han Chinese,"124,505,880,000.00"
1,Indo-Aryan,"87,181,558,344.00"
2,European,"49,487,221,972.00"
3,African,"31,832,512,037.00"
4,Dravidian,"30,271,374,425.00"
5,Mestizo,"15,773,435,494.00"
6,Bengali,"14,677,691,672.00"
7,Russian,"13,185,699,608.00"
8,Japanese,"12,653,421,200.00"
9,Malay,"12,199,355,037.00"


## 4. (a) name and country of longest river: 

In [49]:
# create a dataframe to hold the parsed data values
df_river = pd.DataFrame(columns=('river_name', 'river_id', 'country_code', 'river_length'))

# parse the document
for element in document.iterfind('river'):
      try:

        # retrieve values needed 
        river_name = element.find('name').text
        country_code = element.attrib['country']
        river_id = element.attrib['id']
        river_length = element.find('length').text
        
        # add values to the dataframe
        row = dict(zip(['river_name', 'river_id', 'country_code', 'river_length'], [river_name, river_id, country_code, river_length]))
        row = pd.Series(row)
        row.name = 1
        df_river = df_river.append(row)   

      except:
            pass

# change the data type of numeric columns       
df_river['river_length'] = df_river.river_length.astype(float)        
df_river.head(20)


,river_name,river_id,country_code,river_length
1,Thjorsa,river-Thjorsa,IS,230.00
1,Joekulsa a Fjoellum,river-Joekulsa_a_Fjoellum,IS,206.00
1,Glomma,river-Glomma,N,604.00
1,Lagen,river-Lagen,N,322.00
1,Goetaaelv,river-Goetaaelv,S,93.00
1,Klaraelv,river-Klaraelv,N S,460.00
1,Umeaelv,river-Umeaelv,S,470.00
1,Dalaelv,river-Dalaelv,S,520.00
1,Vaesterdalaelv,river-Vaesterdalaelv,S,320.00
1,Oesterdalaelv,river-Oesterdalaelv,S,241.00


Note: 
-  we need the country name instead of the country code
-  some values have more than one country code: the river crosses man countries


In [50]:
# get country code - country name mapping
# create a dataframe to hold the parsed data values

df_country_code = pd.DataFrame(columns=('country', 'country_code'))

# parse the document
for element in document.iterfind('country'):
    try:
        
        # retrieve values needed 
        country = element.find('name').text
        country_code = element.attrib['car_code']
        
        # add values to the dataframe
        row = dict(zip(['country', 'country_code'], [country, country_code]))
        row = pd.Series(row)
        row.name = 1
        df_country_code = df_country_code.append(row)
        
    except:
        pass

df_country_code.head()

,country,country_code
1,Albania,AL
1,Greece,GR
1,Macedonia,MK
1,Serbia,SRB
1,Montenegro,MNE


In [51]:
# define a fuction to resolve country code(s) into country name(s)

def get_country_names(car_codes):
    """
    resolve country code(s) into country name(s)
    
    input:
        car_codes: string containing the country_code(s)
    
    return: string of equivalent country name(s)
    """

    return df_country_code.loc[df_country_code['country_code'] \
                         .isin(car_codes.split())]['country'] \
                         .str.cat(sep = ' ')


In [52]:
# add a new column and populate it with the country name(s)

df_river['country_name'] = df_river['country_code'].apply(get_country_names)

In [53]:
# sort the data frame based on river_length in descendent order and re-index

df_river.sort_values('river_length', ascending=False, inplace=True)
df_river.reset_index(drop=True, inplace=True)
df_river.head()

,river_name,river_id,country_code,river_length,country_name
0,Amazonas,river-Amazonas,CO BR PE,"6,448.00",Colombia Brazil Peru
1,Jangtse,river-Jangtse,CN,"6,380.00",China
2,Hwangho,river-Hwangho,CN,"4,845.00",China
3,Lena,river-Lena,R,"4,400.00",Russia
4,Zaire,river-Zaire,RCB ZRE,"4,374.00",Congo Zaire


In [54]:
# results: name and country of longest river
df_river[['river_name', 'country_name']].head(1)

,river_name,country_name
0,Amazonas,Colombia Brazil Peru


## 4. (b) name and country of largest lake: 

In [55]:
# create a dataframe to hold the parsed data values
df_lake = pd.DataFrame(columns=('lake_name', 'lake_id', 'country_code', 'lake_area'))

# parse the document
for element in document.iterfind('lake'):
      try:
        
        # retrieve values needed
        lake_name = element.find('name').text
        country_code = element.attrib['country']
        lake_id = element.attrib['id']
        lake_area = element.find('area').text
     
        # add values to the dataframe
        row = dict(zip(['lake_name', 'lake_id', 'country_code', 'lake_area'], [lake_name, lake_id, country_code, lake_area]))
        row = pd.Series(row)
        row.name = 1
        df_lake = df_lake.append(row)   

      except:
            pass

# change the data type of numeric columns
df_lake['lake_area'] = df_lake.lake_area.astype(float)
df_lake.head()

,lake_name,lake_id,country_code,lake_area
1,Inari,lake-Inarisee,SF,"1,040.00"
1,Oulujaervi,lake-Oulujaervi,SF,928.00
1,Kallavesi,lake-Kallavesi,SF,472.00
1,Saimaa,lake-Saimaa,SF,"4,370.00"
1,Paeijaenne,lake-Paeijaenne,SF,"1,118.00"


In [56]:
# add a new column and populate it with the country name(s)

df_lake['country_name'] = df_lake['country_code'].apply(get_country_names)

In [57]:
# sort the data frame based on lake area in descendent order and re-index

df_lake.sort_values('lake_area', ascending=False, inplace=True)
df_lake.reset_index(drop=True, inplace=True)
df_lake.head()

,lake_name,lake_id,country_code,lake_area,country_name
0,Caspian Sea,lake-KaspischesMeer,R AZ KAZ IR TM,"386,400.00",Russia Iran Turkmenistan Azerbaijan Kazakhstan
1,Lake Superior,lake-Lake_Superior,CDN USA,"82,103.00",Canada United States
2,Lake Victoria,lake-Victoriasee,EAT EAK EAU,"68,870.00",Tanzania Kenya Uganda
3,Lake Huron,lake-Lake_Huron,CDN USA,"59,600.00",Canada United States
4,Lake Michigan,lake-Lake_Michigan,USA,"57,800.00",United States


In [58]:
# results:  name and country of largest lake

df_lake[['lake_name', 'country_name']].head(1)

,lake_name,country_name
0,Caspian Sea,Russia Iran Turkmenistan Azerbaijan Kazakhstan


## 4. (c) name and country of airport at highest elevation:

In [59]:
# create a dataframe to hold the parsed data values
df_airport = pd.DataFrame(columns=('airport_name', 'airport_id', 'country_code', 'airport_elevation'))

# parse the document
for element in document.iterfind('airport'):
      try:
        
        # retrieve values needed
        
        airport_name = element.find('name').text
        country_code = element.attrib['country']
        airport_id = element.attrib['iatacode']
        airport_elevation = element.find('elevation').text
    
        # add values to the dataframe
        row = dict(zip(['airport_name', 'airport_id', 'country_code', 'airport_elevation'], [airport_name, airport_id, country_code, airport_elevation]))
        row = pd.Series(row)
        row.name = 1
        df_airport = df_airport.append(row)   

      except:
            pass

# change the data type of numeric columns
df_airport['airport_elevation'] = df_airport.airport_elevation.astype(float)
df_airport.head()

,airport_name,airport_id,country_code,airport_elevation
1,Herat,HEA,AFG,977.00
1,Kabul Intl,KBL,AFG,"1,792.00"
1,Tirana Rinas,TIA,AL,38.00
1,Cheikh Larbi Tebessi,TEE,DZ,811.00
1,Batna Airport,BLJ,DZ,822.00


In [60]:
# add a new column and populate it with the country name(s)

df_airport['country_name'] = df_airport['country_code'].apply(get_country_names)

In [61]:
# sort the data frame based on airport elevation in descendent order and re-index

df_airport.sort_values('airport_elevation', ascending=False, inplace=True)
df_airport.reset_index(drop=True, inplace=True)
df_airport.head()


,airport_name,airport_id,country_code,airport_elevation,country_name
0,El Alto Intl,LPB,BOL,"4,063.00",Bolivia
1,Lhasa-Gonggar,LXA,CN,"4,005.00",China
2,Yushu Batang,YUS,CN,"3,963.00",China
3,Juliaca,JUL,PE,"3,827.00",Peru
4,Teniente Alejandro Velasco Astete Intl,CUZ,PE,"3,311.00",Peru


In [62]:
# results:  name and country of airport at highest elevation

df_airport[['airport_name', 'country_name']].head(1)

,airport_name,country_name
0,El Alto Intl,Bolivia
